# Data Wrangling Personalitics Data

## 0.) Load the data

In [1]:
import pandas as pd
import json
import re
from datetime import datetime
from scipy.stats import mode
import sqlite3
from bs4 import BeautifulSoup
from tqdm import tqdm
from utils import parse_type_personality_cafe, unpack_topic_user, unpack_comments, \
                  unpack_comment_user, html_to_text, parse_type_16personality
from utils import TYPES
import os
from dask import dataframe  as dd

OUTPUT_DIR = r'../output/'
COMMENT_USER_DIR = OUTPUT_DIR + r'df_comment_user_chunks/'
COMMENT_DIR = OUTPUT_DIR + r'df_comment_chunks/'


In [22]:
df_chunks = pd.read_csv('../output/discussion_2.csv', chunksize=4000,
                       usecols=['comment_list', 'url', 'id'])
# List of columns of interest
comment_user_cols = ['id', 'profileUrl', 'avatar', 'gender', 'reputation', 'type']
comment_cols = ['id', 'approvedAtNice', 'answerBody', 'url']

for i, chunk in tqdm(enumerate(df_chunks)):
    # Unpack the comment json 
    chunk_comment = chunk[['comment_list', 'url']].apply(unpack_comments, axis=1)
    chunk_comment = pd.concat(chunk_comment.values)
    
    # Unpack the comment_user json
    chunk_comment_user = chunk_comment[['user', 'id']].apply(unpack_comment_user, axis=1)
    

    # Save into a csv file
    chunk_comment_user[comment_user_cols].to_csv(COMMENT_USER_DIR + 'df_comment_user{}.csv'.format(i), index=False)
    chunk_comment[comment_cols].to_csv(COMMENT_DIR + 'df_comment{}.csv'.format(i), index=False)

32it [49:43, 93.24s/it] 


## 1.) Unpack the topic_user column then merge it into the DataFrame

In [50]:
def get_comment_df():
    tmp_df_ls = []
    for file_user, file_comment in tqdm(zip(os.listdir(COMMENT_USER_DIR), os.listdir(COMMENT_DIR))):
        curr_file_user = os.path.join(COMMENT_USER_DIR, file_user)
        curr_file_comment = os.path.join(COMMENT_DIR, file_comment)

        tmp_df_comments_user = pd.read_csv(curr_file_user)
        tmp_df_comment = pd.read_csv(curr_file_comment)
        tmp_df = pd.merge(tmp_df_comment, tmp_df_comments_user, how='inner', on='id')
        tmp_df_ls.append(tmp_df)
    final_df = pd.concat(tmp_df_ls)
    return final_df

In [51]:
df_merged = get_comment_df()

32it [00:12,  2.56it/s]


In [52]:
df_merged.columns = list(map(lambda x: 'sub-'+ x, df_merged.columns))

In [54]:
df_merged.shape

(1169475, 9)

In [55]:
df_merged.to_csv(OUTPUT_DIR + r'comment_discussion.csv', index=False)

## 3.) Tweak memory usage

In [ ]:
# df_merged['sub-id'] = df_merged['sub-id'].astype('int64')
# df_merged['sub-approved'] = df_merged['sub-approved'].astype('int8')
# df_merged['sub-createdAtDiff'] = df_merged['sub-createdAtDiff'].astype('int32')
# df_merged['sub-reportCount'] = df_merged['sub-reportCount'].astype('int32')
# df_merged['sub-reviewed'] = df_merged['sub-reviewed'].astype('int8')
# df_merged['sub-subCommentCount'] = df_merged['sub-subCommentCount'].astype('int32')
# df_merged['sub-totalVotingScore'] = df_merged['sub-totalVotingScore'].astype('int32')
# df_merged['sub-unavailable'] = df_merged['sub-unavailable'].astype('int8')
# df_merged['sub-upvotedByUser'] = df_merged['sub-upvotedByUser'].astype('bool')

# df_merged['sub-updatedByUser'] = df_merged['sub-updatedByUser'].astype('bool')
# df_merged['sub-unread'] = df_merged['sub-unread'].astype('bool')
# df_merged['sub-reportIgnored'] = df_merged['sub-reportIgnored'].astype('bool')
# df_merged['sub-reportedByUser'] = df_merged['sub-reportedByUser'].astype('bool')
# df_merged['sub-own'] = df_merged['sub-own'].astype('bool')
# df_merged['sub-hasUnreadSubComments'] = df_merged['sub-hasUnreadSubComments'].astype('bool')
# df_merged['sub-hasDisapprovalReason'] = df_merged['sub-hasDisapprovalReason'].astype('bool')
# df_merged['sub-disapprovalReason'] = df_merged['sub-disapprovalReason'].fillna('N/A')
# df_merged['sub-states'] = df_merged['sub-states'].apply(lambda x: json.dumps(x))

In [ ]:
# new_mem_usage = df_merged.memory_usage().sum()
# new_mem_usage

In [ ]:
# memory_saved = (old_mem_usage - new_mem_usage)/1000000

In [ ]:
# print('Total Memory Usage saved:', round(memory_saved, 2), 'mb')

In [ ]:
# # Save the modified DataFrame
# df_merged.to_csv('../output/modified_comment.csv', index=False)

## 4.) Aggregating text posts based on User ID

In [3]:
def get_date(x):
    try:
        return re.sub('\..+', '', x)
    except:
        print(x)

def aggregate_data(source):
    if source == 'personalitycafe':
        conn = sqlite3.connect('../output/project_mbti.db')
        df_db = pd.read_sql('SELECT user_id, user_type, child_text, date, created_at FROM personalitics', con=conn)
        temp_df = pd.read_csv('../output/personality_cafe.csv', usecols=['user_id', 'user_type', 'child_text', 'date', 'created_at'])
        temp_df = pd.concat([temp_df, df_db])
        temp_df = temp_df[~(temp_df.duplicated())]
        temp_df = temp_df[~(temp_df['user_type'].isna())]
        temp_df = temp_df[~(temp_df['child_text'].isna())]
        temp_df['user_type'] = temp_df['user_type'].apply(parse_type_personality_cafe)
        temp_df = temp_df[(temp_df['user_type'].isin(TYPES))]
        has_yesterday = temp_df['date'].str.contains(r'Yesterday|Today')
        temp_df.loc[has_yesterday, 'date'] = temp_df.loc[has_yesterday, 'date'].str.strip('Yesterday ').str.strip('Today ')
        temp_df['created_at'] = temp_df['created_at'].apply(get_date)
        temp_df.loc[has_yesterday, 'date'] = temp_df.loc[has_yesterday, 'created_at'].str.replace(r' \d+:\d+:\d+', '') +  ' ' + temp_df.loc[has_yesterday, 'date'].astype(str)
        temp_df['date'] = pd.to_datetime(temp_df['date'])
        temp_df['dow'] = temp_df['date'].copy()
        temp_df = temp_df[['user_id', 'child_text', 'date', 'dow', 'user_type']].groupby(['user_id', 'user_type'])
        temp_df = temp_df.agg({'date': lambda x: x.dt.hour.median(),
                               'child_text': '|||'.join,
                               'dow': lambda z: mode(z.dt.dayofweek).mode[0]}).reset_index()
        temp_df.to_csv('../output/aggregated/data_personalitycafe.csv', index=False)
        
    elif source == '16personalities_discussion_comments':
        chunks = pd.read_csv('../output/comment_discussion.csv', 
                             usecols=['sub-approvedAtNice', 'sub-answerBody', 'sub-type', 'sub-profileUrl'],
                             chunksize=4096)
        temp_ls = []
        for chunk in tqdm(chunks):
            temp_df = chunk.drop(axis=1, index=chunk[chunk['sub-type'].isna()].index)
            temp_df = temp_df.drop(axis=1, index=temp_df[temp_df['sub-answerBody'].isna()].index)
            temp_df['sub-answerBody'] = temp_df['sub-answerBody'].apply(html_to_text)
            temp_df['sub-approvedAtNice'] = pd.to_datetime(temp_df['sub-approvedAtNice'])
            temp_df['sub-dow'] = temp_df['sub-approvedAtNice'].copy()
            temp_df = temp_df[['sub-profileUrl', 'sub-answerBody', 'sub-approvedAtNice', 'sub-dow', 'sub-type']]
            temp_ls.append(temp_df)
            
        temp_df = dd.from_pandas(pd.concat(temp_ls), 512, sort=False)
        temp_df = temp_df.groupby(['sub-profileUrl', 'sub-type'])
        temp_df = temp_df.agg({'sub-answerBody': '|||'.join, 
                               'sub-approvedAtNice': lambda x: x.dt.hour.median(),
                               'sub-dow': lambda z: mode(z.dt.dayofweek).mode[0]}).compute().reset_index()
        temp_df.to_csv('../output/aggregated/data_discussion_16personalities.csv', index=False)
        
    elif source == '16personalities_pub_comments':
        temp_df = pd.read_csv('../output/sixteenpersonalities.csv')
        temp_df = temp_df[~(temp_df['child_text'].isna())]
        temp_df = temp_df[~(temp_df['user_type'].isna())]
        temp_df['user_type'] = temp_df['user_type'].apply(parse_type_16personality)
        temp_df = temp_df[(temp_df['user_type'].isin(TYPES))]
        temp_df = temp_df[['user_id', 'child_text', 'user_type']].groupby(['user_id', 'user_type']).agg({'child_text': ' '.join}).reset_index()[['child_text', 'user_type']]
        temp_df.to_csv('../output/aggregated/data_pub_16personalities.csv', index=False)

In [3]:
aggregate_data('personalitycafe')

In [4]:
aggregate_data('16personalities_discussion_comments')

286it [02:48,  1.70it/s]


MemoryError: 

In [14]:
aggregate_data('16personalities_pub_comments')

In [11]:
temp_df = pd.read_csv('../output/comment_discussion.csv', nrows=2000)

In [2]:
chunks = pd.read_csv('../output/comment_discussion.csv', 
                     usecols=['sub-approvedAtNice', 'sub-answerBody', 'sub-type', 'sub-profileUrl'],
                     chunksize=4096)
temp_ls = []
for chunk in tqdm(chunks):
    temp_df = chunk.drop(axis=1, index=chunk[chunk['sub-type'].isna()].index)
    temp_df = temp_df.drop(axis=1, index=temp_df[temp_df['sub-answerBody'].isna()].index)
    temp_df['sub-answerBody'] = temp_df['sub-answerBody'].apply(html_to_text)
    temp_df['sub-approvedAtNice'] = pd.to_datetime(temp_df['sub-approvedAtNice'])
    temp_df['sub-dow'] = temp_df['sub-approvedAtNice'].copy()
    temp_df = temp_df[['sub-profileUrl', 'sub-answerBody', 'sub-approvedAtNice', 'sub-dow', 'sub-type']]
    temp_ls.append(temp_df)

temp_df = pd.concat(temp_ls)
temp_df = temp_df.set_index(['sub-profileUrl', 'sub-type'])
temp_df['sub-approvedAtNice'] = temp_df['sub-approvedAtNice'].dt.dayofweek.astype('int')
temp_df['sub-dow'] = temp_df['sub-dow'].dt.hour.astype('int')
temp_df.groupby(temp_df.index).agg({'sub-answerBody': '|||'.join, 
                                    'sub-approvedAtNice': lambda x: x.median(),
                                    'sub-dow': lambda z: mode(z).mode[0]}, observed=True)
# temp_df.to_csv('../output/aggregated/data_discussion_16personalities.csv', index=False)

286it [02:46,  1.71it/s]


In [14]:
temp_df.to_csv('../output/discussion_comments_16personalities.csv', index=True)

In [2]:
temp_df = pd.read_csv('../output/aggregated/data_personalitycafe.csv')

In [5]:
temp_df['child_text'].shape

(22403,)